In [ ]:
!pip install transformers torch spacy

In [ ]:
import torch
from torch.utils.data import DataLoader, TensorDataset, random_split
from transformers import BertTokenizer, BertForSequenceClassification, AdamW
import numpy as np
from sklearn.metrics import accuracy_score, classification_report
from torch.nn import BCEWithLogitsLoss
import spacy
import pandas as pd
import torch.nn as nn

In [ ]:
class BertPreprocessor:
    def __init__(self, model_name='bert-base-uncased', max_length=256):
        self.tokenizer = BertTokenizer.from_pretrained(model_name)
        self.max_length = max_length

    def preprocess(self, texts):
        encoding = self.tokenizer.batch_encode_plus(
            texts,
            add_special_tokens=True,
            max_length=self.max_length,
            return_attention_mask=True,
            padding='max_length',
            truncation=True,
            return_tensors='pt'
        )
        return encoding['input_ids'], encoding['attention_mask']

In [ ]:
class EntityEmbedder:
    def __init__(self, glove_file_path):
        self.embeddings_index = {}
        self.load_glove_embeddings(glove_file_path)

    def load_glove_embeddings(self, file_path):
        with open(file_path, 'r', encoding='utf-8') as f:
            for line in f:
                values = line.split()
                word = values[0]
                coefs = np.asarray(values[1:], dtype='float32')
                self.embeddings_index[word] = coefs
        print(f"Loaded {len(self.embeddings_index)} word vectors.")

    def get_embedding(self, word):
        return self.embeddings_index.get(word, np.zeros(50))

    def create_embedding_matrix(self, unique_entities):
        embedding_matrix = np.zeros((len(unique_entities), 50))
        for i, entity in enumerate(unique_entities):
            embedding_matrix[i] = self.get_embedding(entity)
        return embedding_matrix

In [ ]:
class BertModel:
    def __init__(self, model_name='bert-base-uncased', entity_embedding_matrix=None):
        self.model = BertForSequenceClassification.from_pretrained(model_name, num_labels=2, output_hidden_states=True)
        self.device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
        self.model.to(self.device)

        if entity_embedding_matrix is not None:
            self.entity_embedding_matrix = torch.tensor(entity_embedding_matrix, dtype=torch.float).to(self.device)
            self.entity_embedding_layer = nn.Embedding.from_pretrained(self.entity_embedding_matrix)

            bert_output_size = 768
            entity_embedding_size = entity_embedding_matrix.shape[1]
            self.concatenated_layer_size = bert_output_size + entity_embedding_size

            self.classifier = nn.Linear(self.concatenated_layer_size, 2).to(self.device)

    def train(self, input_ids, attention_masks, labels, entity_indices, batch_size=16, epochs=4):
        dataset = TensorDataset(input_ids, attention_masks, labels, entity_indices)
        train_size = int(0.9 * len(dataset))
        val_size = len(dataset) - train_size
        train_dataset, val_dataset = random_split(dataset, [train_size, val_size])

        train_dataloader = DataLoader(train_dataset, batch_size=batch_size, shuffle=True)
        val_dataloader = DataLoader(val_dataset, batch_size=batch_size, shuffle=False)

        optimizer = AdamW(self.model.parameters(), lr=2e-5)
        loss_fct = BCEWithLogitsLoss()

        for epoch in range(epochs):
            self.model.train()
            total_loss = 0
            for batch in train_dataloader:
                b_input_ids, b_input_mask, b_labels, b_entity_indices = [t.to(self.device) for t in batch]
                self.model.zero_grad()

                outputs = self.model(b_input_ids, token_type_ids=None, attention_mask=b_input_mask)
                bert_output = outputs.hidden_states[-1][:, 0, :]

                entity_embeddings = self.entity_embedding_layer(b_entity_indices)
                entity_embeddings = torch.max(entity_embeddings, dim=1).values

                concatenated_output = torch.cat((bert_output, entity_embeddings), dim=1)
                logits = self.classifier(concatenated_output)

                b_labels = b_labels.unsqueeze(1)
                b_labels = torch.cat((1 - b_labels, b_labels), dim=1)

                loss = loss_fct(logits, b_labels.float())
                total_loss += loss.item()
                loss.backward()
                optimizer.step()

            avg_train_loss = total_loss / len(train_dataloader)
            print(f"Epoch {epoch+1}/{epochs} - Loss: {avg_train_loss:.2f}")

            self.model.eval()
            total_eval_accuracy = 0
            for batch in val_dataloader:
                b_input_ids, b_input_mask, b_labels, b_entity_indices = [t.to(self.device) for t in batch]
                with torch.no_grad():
                    outputs = self.model(b_input_ids, token_type_ids=None, attention_mask=b_input_mask)
                    bert_output = outputs.hidden_states[-1][:, 0, :]

                    entity_embeddings = self.entity_embedding_layer(b_entity_indices)
                    entity_embeddings = torch.max(entity_embeddings, dim=1).values

                    concatenated_output = torch.cat((bert_output, entity_embeddings), dim=1)
                    logits = self.classifier(concatenated_output)

                    logits = logits.detach().cpu().numpy()
                    label_ids = b_labels.to('cpu').numpy()

                    predictions = np.argmax(logits, axis=1)
                    total_eval_accuracy += accuracy_score(label_ids, predictions)

            avg_val_accuracy = total_eval_accuracy / len(val_dataloader)
            print(f"Validation Accuracy: {avg_val_accuracy:.2f}")


    def evaluate(self, input_ids, attention_masks, labels, entity_indices, batch_size=16):
        test_dataset = TensorDataset(input_ids, attention_masks, labels, entity_indices)
        test_dataloader = DataLoader(test_dataset, batch_size=batch_size, shuffle=False)

        self.model.eval()
        total_eval_accuracy = 0
        predictions, true_labels = [], []

        for batch in test_dataloader:
            b_input_ids, b_input_mask, b_labels, b_entity_indices = [t.to(self.device) for t in batch]
            with torch.no_grad():
                outputs = self.model(b_input_ids, token_type_ids=None, attention_mask=b_input_mask)
                bert_output = outputs.hidden_states[-1][:, 0, :]  # Use the last hidden state of the [CLS] token

                entity_embeddings = self.entity_embedding_layer(b_entity_indices)
                entity_embeddings = torch.max(entity_embeddings, dim=1).values  # Or use another pooling method

                concatenated_output = torch.cat((bert_output, entity_embeddings), dim=1)
                logits = self.classifier(concatenated_output)

                logits = logits.detach().cpu().numpy()
                label_ids = b_labels.to('cpu').numpy()

                batch_predictions = np.argmax(logits, axis=1)
                predictions.extend(batch_predictions)
                true_labels.extend(label_ids)

        print("Accuracy:", accuracy_score(true_labels, predictions))
        print("Classification Report:")
        print(classification_report(true_labels, predictions))

In [ ]:
from google.colab import files
uploaded = files.upload()


Saving train_data.tsv to train_data.tsv
Saving glove.6B.50d.txt to glove.6B.50d.txt
Saving test_data.tsv to test_data.tsv
Saving Validation_data.tsv to Validation_data.tsv


In [ ]:
def main():
    # Helper function for entity embedding
    def generate_entity_indices(df, entity_to_index):
        entity_indices = []
        for _, row in df.iterrows():
            indices = [entity_to_index.get(row[col], 0) for col in ['speaker', 'JobTitle', 'subject', 'Party']]
            entity_indices.append(indices)
        return torch.tensor(entity_indices)

    # Load the datasets
    df_train = pd.read_csv('train_data.tsv', sep='\t')
    df_test = pd.read_csv('test_data.tsv', sep='\t')
    df_validate = pd.read_csv('Validation_data.tsv', sep='\t')

    # Define the column names
    columns = ['index', 'id', 'label', 'statement', 'subject', 'speaker', 'JobTitle', 'State', 'Party', 'BTC', 'FC', 'HT', 'MT', 'POF', 'context', 'justification']
    df_train.columns = columns
    df_test.columns = columns
    df_validate.columns = columns

    # Label mapping
    label_mapping = {'true': 1, 'half-true': 1, 'mostly-true': 1, 'false': 0, 'pants-fire': 0, 'barely-true': 0}
    df_train['label'] = df_train['label'].map(label_mapping)
    df_test['label'] = df_test['label'].map(label_mapping)
    df_validate['label'] = df_validate['label'].map(label_mapping)

    # Entity Embedding for all datasets
    entity_embedder = EntityEmbedder('glove.6B.50d.txt')
    unique_entities = set(df_train['speaker'].tolist() + df_train['JobTitle'].tolist() + df_train['subject'].tolist() + df_train['Party'].tolist())
    embedding_matrix = entity_embedder.create_embedding_matrix(list(unique_entities))
    entity_to_index = {entity: i for i, entity in enumerate(unique_entities)}

    # Entity Embedding and Text Preprocessing for each dataset
    bert_preprocessor = BertPreprocessor()

    def preprocess_dataset(df):
        entity_indices = generate_entity_indices(df, entity_to_index)
        texts = df['statement'].tolist()
        input_ids, attention_masks = bert_preprocessor.preprocess(texts)
        y_bert = torch.tensor(df['label'].values)
        return input_ids, attention_masks, y_bert, entity_indices

    input_ids_train, attention_masks_train, y_bert_train, entity_indices_train = preprocess_dataset(df_train)
    input_ids_test, attention_masks_test, y_bert_test, entity_indices_test = preprocess_dataset(df_test)
    input_ids_validate, attention_masks_validate, y_bert_validate, entity_indices_validate = preprocess_dataset(df_validate)

    # Initialize and train the BERT model with entity embeddings for the Train Dataset
    bert_model = BertModel('bert-base-uncased', embedding_matrix)
    bert_model.train(input_ids_train, attention_masks_train, y_bert_train, entity_indices_train)
    print("Finished Training")

    # Evaluate on Test and Validation Datasets
    print("Testing")
    bert_model.evaluate(input_ids_test, attention_masks_test, y_bert_test, entity_indices_test)
    print("Validation")
    bert_model.evaluate(input_ids_validate, attention_masks_validate, y_bert_validate, entity_indices_validate)
    print("Training Test")
    bert_model.evaluate(input_ids_train, attention_masks_train, y_bert_train, entity_indices_train)

main()


Loaded 400000 word vectors.


tokenizer_config.json:   0%|          | 0.00/28.0 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/232k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/466k [00:00<?, ?B/s]

config.json:   0%|          | 0.00/570 [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/440M [00:00<?, ?B/s]

Some weights of BertForSequenceClassification were not initialized from the model checkpoint at bert-base-uncased and are newly initialized: ['classifier.bias', 'classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.
/usr/local/lib/python3.10/dist-packages/transformers/optimization.py:411: FutureWarning: This implementation of AdamW is deprecated and will be removed in a future version. Use the PyTorch implementation torch.optim.AdamW instead, or set `no_deprecation_warning=True` to disable this warning
  warnings.warn(


Epoch 1/4 - Loss: 0.66
Validation Accuracy: 0.68
Epoch 2/4 - Loss: 0.59
Validation Accuracy: 0.65
Epoch 3/4 - Loss: 0.40
Validation Accuracy: 0.60
Epoch 4/4 - Loss: 0.17
Validation Accuracy: 0.61
Finished Training
Testing
Accuracy: 0.8341232227488151
Classification Report:
              precision    recall  f1-score   support

           0       0.81      0.81      0.81       553
           1       0.85      0.86      0.85       713

    accuracy                           0.83      1266
   macro avg       0.83      0.83      0.83      1266
weighted avg       0.83      0.83      0.83      1266

Validation
Accuracy: 0.8371005455962588
Classification Report:
              precision    recall  f1-score   support

           0       0.84      0.81      0.83       615
           1       0.83      0.86      0.85       668

    accuracy                           0.84      1283
   macro avg       0.84      0.84      0.84      1283
weighted avg       0.84      0.84      0.84      1283

Training 